In [1]:
import sys
import os

# 将当前notebook所在目录（即本文件的上层目录）加入到sys.path中
notebook_dir = "/home/muwang/Documents/GitHub/TomoPANDA-pick"
if notebook_dir not in sys.path:
    sys.path.insert(0, notebook_dir)


In [2]:
from utils.io_dynamo import *


In [3]:
project_dir = '/home/muwang/Documents/GitHub/TomoPANDA-pick'
vll_path = os.path.join(project_dir, 'test_data/tomograms_bin8.vll')
tbl_path = os.path.join(project_dir, 'test_data/particles.tbl')
rl_particles_path = os.path.join(project_dir, 'test_data/particles.star')
output_dir = os.path.join(project_dir, 'output')

In [4]:
# Test 1: convert_euler function
print("=== Testing convert_euler function ===")

# Test data: ZYZ angles in degrees
test_angles_zyz = np.array([
    [0, 0, 0],
    [30, 45, 60],
    [90, 90, 90]
])

print("Input ZYZ angles (degrees):")
print(test_angles_zyz)

# Convert ZYZ to ZXZ
converted_angles = convert_euler(test_angles_zyz, src_convention='ZYZ', dst_convention='ZXZ', degrees=True)
print("\nConverted to ZXZ angles (degrees):")
print(converted_angles)

# Convert back to verify
back_converted = convert_euler(converted_angles, src_convention='ZXZ', dst_convention='ZYZ', degrees=True)
print("\nBack-converted to ZYZ angles (degrees):")
print(back_converted)

print(f"\nConversion accuracy (should be close to original): {np.allclose(test_angles_zyz, back_converted)}")


=== Testing convert_euler function ===
Input ZYZ angles (degrees):
[[ 0  0  0]
 [30 45 60]
 [90 90 90]]

Converted to ZXZ angles (degrees):
[[  0.   0.   0.]
 [120.  45. -30.]
 [180.  90.   0.]]

Back-converted to ZYZ angles (degrees):
[[ 0.  0.  0.]
 [30. 45. 60.]
 [90. 90. 90.]]

Conversion accuracy (should be close to original): True


/home/muwang/miniforge3/envs/tomopanda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3579: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
# Test 2: read_vll_to_df function
print("=== Testing read_vll_to_df function ===")

# Read the VLL file
vll_df = read_vll_to_df(vll_path)
print(f"Number of tomograms in VLL file: {len(vll_df)}")
print("\nFirst 5 tomogram names:")
print(vll_df.head())

print(f"\nVLL file path: {vll_path}")
print("VLL file exists:", os.path.exists(vll_path))


=== Testing read_vll_to_df function ===
Number of tomograms in VLL file: 63

First 5 tomogram names:
  rlnMicrographName
0     tomo_01_bin_8
1     tomo_02_bin_8
2     tomo_03_bin_8
3     tomo_04_bin_8
4     tomo_05_bin_8

VLL file path: /home/muwang/Documents/GitHub/TomoPANDA-pick/test_data/tomograms_bin8.vll
VLL file exists: True


In [6]:
# Test 3: create_dynamo_table function
print("=== Testing create_dynamo_table function ===")

# Create test coordinates (3 particles)
test_coordinates = np.array([
    [100.3, 200.45, 300.6],
    [150.7, 250.8, 350.9],
    [200.1, 300.2, 400.3]
])

# Create test ZYZ angles (degrees)
test_angles_zyz = np.array([
    [0.3, 0.45, 0.6],
    [30.7, 45.8, 60.9994],
    [90.1, 90.2, 90.3]
])

# Create test micrograph names
test_micrograph_names = ['tomo1', 'tomo1', 'tomo2']

# Create Dynamo table
dynamo_df = create_dynamo_table(
    coordinates=test_coordinates,
    angles_zyz=test_angles_zyz,
    micrograph_names=test_micrograph_names,
    output_file=os.path.join(output_dir, 'test_particles.tbl')
)

print(f"Created Dynamo table with {len(dynamo_df)} particles")
print("\nTable columns:")
print(dynamo_df.columns.tolist())

print("\nFirst few rows:")
print(dynamo_df.head())

print(f"\nOutput file created: {os.path.join(output_dir, 'test_particles.tbl')}")
print("File exists:", os.path.exists(os.path.join(output_dir, 'test_particles.tbl')))


=== Testing create_dynamo_table function ===
Created Dynamo table with 3 particles

Table columns:
['tag', 'aligned', 'averaged', 'dx', 'dy', 'dz', 'tdrot', 'tilt', 'narot', 'cc', 'cc2', 'cpu', 'ftype', 'ymintilt', 'ymaxtilt', 'xmintilt', 'xmaxtilt', 'fs1', 'fs2', 'tomo', 'reg', 'class', 'annotation', 'x', 'y', 'z', 'dshift', 'daxis', 'dnarot', 'dcc', 'otag', 'npar', 'col33', 'ref', 'sref']

First few rows:
   tag  aligned  averaged   dx   dy   dz  tdrot   tilt    narot   cc  ...  \
0  1.0      1.0       0.0  0.0  0.0  0.0   90.3   0.45 -89.4000  0.0  ...   
1  2.0      1.0       0.0  0.0  0.0  0.0  120.7  45.80 -29.0006  0.0  ...   
2  3.0      1.0       0.0  0.0  0.0  0.0 -179.9  90.20   0.3000  0.0  ...   

       z  dshift  daxis  dnarot  dcc  otag  npar  col33  ref  sref  
0  300.6     0.0    0.0     0.0  0.0   0.0   0.0    0.0  1.0   0.0  
1  350.9     0.0    0.0     0.0  0.0   0.0   0.0    0.0  1.0   0.0  
2  400.3     0.0    0.0     0.0  0.0   0.0   0.0    0.0  1.0   0.0  

[3 

In [7]:
# Test 4: read_dynamo_tbl function (without VLL mapping)
print("=== Testing read_dynamo_tbl function (without VLL) ===")

# Read the Dynamo table without VLL mapping
dynamo_df_read = read_dynamo_tbl(tbl_path)
print(f"Read Dynamo table with {len(dynamo_df_read)} particles")
print("\nTable columns:")
print(dynamo_df_read.columns.tolist())

print("\nFirst few rows:")
print(dynamo_df_read.head())

# Check if coordinates match
print(f"\nOriginal coordinates:")
print(test_coordinates)
print(f"Read coordinates (x, y, z):")
print(dynamo_df_read[['x', 'y', 'z']].values)


=== Testing read_dynamo_tbl function (without VLL) ===
Read Dynamo table with 3188 particles

Table columns:
['tag', 'aligned', 'averaged', 'dx', 'dy', 'dz', 'tdrot', 'tilt', 'narot', 'cc', 'cc2', 'cpu', 'ftype', 'ymintilt', 'ymaxtilt', 'xmintilt', 'xmaxtilt', 'fs1', 'fs2', 'tomo', 'reg', 'class', 'annotation', 'x', 'y', 'z', 'dshift', 'daxis', 'dnarot', 'dcc', 'otag', 'npar', 'col33', 'ref', 'sref']

First few rows:
       tag  aligned  averaged       dx        dy        dz    tdrot    tilt  \
0  47898.0      1.0       1.0 -0.85729 -0.017691 -0.918880  128.000  93.000   
1  47857.0      1.0       1.0  0.19765  0.272680 -0.065138  171.780  59.913   
2  47797.0      1.0       1.0 -0.34744 -1.212500 -0.696340 -111.970  55.328   
3  47790.0      1.0       1.0 -0.30974 -0.857800 -1.984100  -42.852  97.030   
4  47745.0      1.0       1.0 -0.71829  1.612300 -2.325700   99.304  70.741   

     narot        cc  ...      z  dshift  daxis  dnarot  dcc  otag  npar  \
0 -179.210  0.089729  ...  1

In [8]:
# Test 5: read_dynamo_tbl function (with VLL mapping)
print("=== Testing read_dynamo_tbl function (with VLL mapping) ===")

# Read the Dynamo table with VLL mapping
dynamo_df_with_vll = read_dynamo_tbl(tbl_path, vll_path=vll_path)
print(f"Read Dynamo table with VLL mapping: {len(dynamo_df_with_vll)} particles")
print("\nTable columns:")
print(dynamo_df_with_vll.columns.tolist())

print("\nFirst few rows:")
print(dynamo_df_with_vll.head())

# Check micrograph names
print(f"\nMicrograph names in the table:")
print(dynamo_df_with_vll['rlnMicrographName'].unique())


=== Testing read_dynamo_tbl function (with VLL mapping) ===
Read Dynamo table with VLL mapping: 3188 particles

Table columns:
['tag', 'aligned', 'averaged', 'dx', 'dy', 'dz', 'tdrot', 'tilt', 'narot', 'cc', 'cc2', 'cpu', 'ftype', 'ymintilt', 'ymaxtilt', 'xmintilt', 'xmaxtilt', 'fs1', 'fs2', 'rlnMicrographName', 'reg', 'class', 'annotation', 'x', 'y', 'z', 'dshift', 'daxis', 'dnarot', 'dcc', 'otag', 'npar', 'col33', 'ref', 'sref']

First few rows:
       tag  aligned  averaged       dx        dy        dz    tdrot    tilt  \
0  47898.0      1.0       1.0 -0.85729 -0.017691 -0.918880  128.000  93.000   
1  47857.0      1.0       1.0  0.19765  0.272680 -0.065138  171.780  59.913   
2  47797.0      1.0       1.0 -0.34744 -1.212500 -0.696340 -111.970  55.328   
3  47790.0      1.0       1.0 -0.30974 -0.857800 -1.984100  -42.852  97.030   
4  47745.0      1.0       1.0 -0.71829  1.612300 -2.325700   99.304  70.741   

     narot        cc  ...      z  dshift  daxis  dnarot  dcc  otag  npar 

In [9]:
# Test 6: dynamo_df_to_relion function
print("=== Testing dynamo_df_to_relion function ===")

# Convert Dynamo DataFrame to RELION format
relion_df = dynamo_df_to_relion(dynamo_df_with_vll, bin_scalar=8.0)

print(f"Converted to RELION format: {len(relion_df)} particles")
print("\nRELION DataFrame columns:")
print(relion_df.columns.tolist())

print("\nFirst few rows:")
print(relion_df.head())

# Check coordinate scaling
print(f"\nOriginal coordinates (Dynamo):")
print(dynamo_df_with_vll[['x', 'y', 'z']].head().values)
print(f"RELION coordinates (scaled by 8x):")
print(relion_df[['rlnCoordinateX', 'rlnCoordinateY', 'rlnCoordinateZ']].head().values)

# Check angle conversion
print(f"\nOriginal angles (Dynamo ZXZ):")
print(dynamo_df_with_vll[['tdrot', 'tilt', 'narot']].head().values)
print(f"RELION angles (ZYZ):")
print(relion_df[['rlnAngleRot', 'rlnAngleTilt', 'rlnAnglePsi']].head().values)


=== Testing dynamo_df_to_relion function ===
Using rlnMicrographName column
Converted to RELION format: 3188 particles

RELION DataFrame columns:
['rlnCoordinateX', 'rlnCoordinateY', 'rlnCoordinateZ', 'rlnAngleRot', 'rlnAngleTilt', 'rlnAnglePsi', 'rlnOriginXAngst', 'rlnOriginYAngst', 'rlnOriginZAngst', 'rlnMicrographName']

First few rows:
   rlnCoordinateX  rlnCoordinateY  rlnCoordinateZ  rlnAngleRot  rlnAngleTilt  \
0          2116.0          1476.0          1268.0       38.000        93.000   
1          1060.0          1004.0          1324.0       81.780        59.913   
2           212.0          2876.0          1364.0      158.030        55.328   
3          1260.0          1124.0          1260.0     -132.852        97.030   
4          3340.0          1684.0           260.0        9.304        70.741   

   rlnAnglePsi  rlnOriginXAngst  rlnOriginYAngst  rlnOriginZAngst  \
0      -89.210              0.0              0.0              0.0   
1     -101.610              0.0        

In [13]:
relion_df

,rlnCoordinateX,rlnCoordinateY,rlnCoordinateZ,rlnAngleRot,rlnAngleTilt,rlnAnglePsi,rlnOriginXAngst,rlnOriginYAngst,rlnOriginZAngst,rlnMicrographName
0,2116.0,1476.0,1268.0,38.000,93.000,-89.210,0.0,0.0,0.0,tomo_63_bin_8
1,1060.0,1004.0,1324.0,81.780,59.913,-101.610,0.0,0.0,0.0,tomo_63_bin_8
2,212.0,2876.0,1364.0,158.030,55.328,-124.190,0.0,0.0,0.0,tomo_63_bin_8
3,1260.0,1124.0,1260.0,-132.852,97.030,48.535,0.0,0.0,0.0,tomo_63_bin_8
4,3340.0,1684.0,260.0,9.304,70.741,-32.320,0.0,0.0,0.0,tomo_63_bin_8
...,...,...,...,...,...,...,...,...,...,...
3183,2156.0,2244.0,1076.0,22.510,74.550,-49.500,0.0,0.0,0.0,tomo_01_bin_8
3184,956.0,2524.0,1012.0,-164.664,97.420,109.619,0.0,0.0,0.0,tomo_01_bin_8
3185,460.0,2540.0,1068.0,18.380,98.480,-154.190,0.0,0.0,0.0,tomo_01_bin_8
3186,2220.0,2532.0,916.0,8.000,99.000,-152.220,0.0,0.0,0.0,tomo_01_bin_8


In [10]:
relion_df['rlnMicrographName'].unique()

array(['tomo_63_bin_8', 'tomo_59_bin_8', 'tomo_58_bin_8', 'tomo_56_bin_8',
       'tomo_54_bin_8', 'tomo_53_bin_8', 'tomo_52_bin_8', 'tomo_50_bin_8',
       'tomo_49_bin_8', 'tomo_47_bin_8', 'tomo_46_bin_8', 'tomo_45_bin_8',
       'tomo_44_bin_8', 'tomo_43_bin_8', 'tomo_41_bin_8', 'tomo_40_bin_8',
       'tomo_39_bin_8', 'tomo_38_bin_8', 'tomo_35_bin_8', 'tomo_34_bin_8',
       'tomo_33_bin_8', 'tomo_32_bin_8', 'tomo_31_bin_8', 'tomo_30_bin_8',
       'tomo_29_bin_8', 'tomo_28_bin_8', 'tomo_27_bin_8', 'tomo_25_bin_8',
       'tomo_24_bin_8', 'tomo_23_bin_8', 'tomo_22_bin_8', 'tomo_21_bin_8',
       'tomo_20_bin_8', 'tomo_17_bin_8', 'tomo_16_bin_8', 'tomo_15_bin_8',
       'tomo_14_bin_8', 'tomo_13_bin_8', 'tomo_11_bin_8', 'tomo_10_bin_8',
       'tomo_09_bin_8', 'tomo_08_bin_8', 'tomo_07_bin_8', 'tomo_06_bin_8',
       'tomo_04_bin_8', 'tomo_03_bin_8', 'tomo_02_bin_8', 'tomo_01_bin_8'],
      dtype=object)

In [11]:
import starfile

starfile.write(relion_df, output_dir + '/particles.star')




In [12]:
# Test 7: Integration test - Complete workflow
print("=== Integration Test: Complete Dynamo to RELION Workflow ===")

# Step 1: Read existing VLL file
print("Step 1: Reading VLL file...")
vll_df_integration = read_vll_to_df(vll_path)
print(f"Found {len(vll_df_integration)} tomograms")

# Step 2: Create a more realistic Dynamo table with multiple particles
print("\nStep 2: Creating realistic Dynamo table...")
# Create coordinates for multiple particles across different tomograms
num_particles = 10
coordinates = np.random.randint(50, 500, size=(num_particles, 3))
angles_zyz = np.random.uniform(0, 360, size=(num_particles, 3))
micrograph_names = [f"tomo{i%3+1}" for i in range(num_particles)]  # Distribute across 3 tomograms

# Create the table
integration_df = create_dynamo_table(
    coordinates=coordinates,
    angles_zyz=angles_zyz,
    micrograph_names=micrograph_names,
    output_file='output/integration_test.tbl'
)

print(f"Created table with {len(integration_df)} particles")

# Step 3: Read it back with VLL mapping
print("\nStep 3: Reading back with VLL mapping...")
read_back_df = read_dynamo_tbl('output/integration_test.tbl', vll_path=vll_path)
print(f"Read back {len(read_back_df)} particles")

# Step 4: Convert to RELION format
print("\nStep 4: Converting to RELION format...")
final_relion_df = dynamo_df_to_relion(read_back_df, bin_scalar=8.0)
print(f"Final RELION format: {len(final_relion_df)} particles")

print("\nFinal RELION DataFrame sample:")
print(final_relion_df.head())


# save to new particles.star
import starfile
starfile.write(final_relion_df, output_dir + '/particles.star')

print(f"\nIntegration test completed successfully!")
print(f"Output files created:")
print(f"- output/integration_test.tbl")
print(f"- {tbl_path}")


=== Integration Test: Complete Dynamo to RELION Workflow ===
Step 1: Reading VLL file...
Found 63 tomograms

Step 2: Creating realistic Dynamo table...
Created table with 10 particles

Step 3: Reading back with VLL mapping...
Read back 10 particles

Step 4: Converting to RELION format...
Using rlnMicrographName column
Final RELION format: 10 particles

Final RELION DataFrame sample:
   rlnCoordinateX  rlnCoordinateY  rlnCoordinateZ  rlnAngleRot  rlnAngleTilt  \
0          1824.0          2064.0          2128.0     -25.4517      118.5070   
1          3736.0          2200.0          3944.0    -114.8740      175.0030   
2          1312.0          3080.0          2088.0     128.6310       36.8658   
3          2112.0          1968.0          2944.0     101.3790      116.3350   
4          1400.0          3752.0          1792.0    -113.4957      169.6830   

   rlnAnglePsi  rlnOriginXAngst  rlnOriginYAngst  rlnOriginZAngst  \
0      18.7432              0.0              0.0              0.

In [17]:
# from star to tbl

starfile_particle = '/home/muwang/Documents/projects/IR/20230518IR_mCherry/merged_particles_changeTomoName.star'
df =relion_star_to_dynamo_tbl(starfile_particle, 6.72,tomogram_size=(999,999,499), output_file='output/particles_changeTomoName_bin8.tbl')

In [18]:
df.keys()



Index(['tag', 'aligned', 'averaged', 'dx', 'dy', 'dz', 'tdrot', 'tilt',
       'narot', 'cc', 'cc2', 'cpu', 'ftype', 'ymintilt', 'ymaxtilt',
       'xmintilt', 'xmaxtilt', 'fs1', 'fs2', 'tomo', 'reg', 'class',
       'annotation', 'x', 'y', 'z', 'dshift', 'daxis', 'dnarot', 'dcc', 'otag',
       'npar', 'col33', 'ref', 'sref'],
      dtype='object')

In [19]:
df

,tag,aligned,averaged,dx,dy,dz,tdrot,tilt,narot,cc,...,z,dshift,daxis,dnarot,dcc,otag,npar,col33,ref,sref
0,1.0,1.0,0.0,0.0,0.0,0.0,83.076920,84.113158,-18.461538,0.0,...,243.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2.0,1.0,0.0,0.0,0.0,0.0,65.769230,75.142834,-101.538460,0.0,...,129.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,3.0,1.0,0.0,0.0,0.0,0.0,48.461540,90.000000,-69.230769,0.0,...,163.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,4.0,1.0,0.0,0.0,0.0,0.0,-13.846150,95.886842,-78.461538,0.0,...,219.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,5.0,1.0,0.0,0.0,0.0,0.0,76.153850,84.113158,-50.769231,0.0,...,177.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58529,58530.0,1.0,0.0,0.0,0.0,0.0,-96.923077,90.000000,-124.615390,0.0,...,313.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
58530,58531.0,1.0,0.0,0.0,0.0,0.0,-121.153846,104.857166,96.923080,0.0,...,243.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
58531,58532.0,1.0,0.0,0.0,0.0,0.0,-100.384615,92.939534,13.846154,0.0,...,371.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
58532,58533.0,1.0,0.0,0.0,0.0,0.0,124.615385,59.148114,92.307690,0.0,...,31.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [20]:
df_star = starfile.read(starfile_particle)
df_star.keys()

Index(['rlnCenteredCoordinateXAngst', 'rlnCenteredCoordinateYAngst',
       'rlnCenteredCoordinateZAngst', 'rlnAngleRot', 'rlnAngleTilt',
       'rlnAnglePsi', 'rlnLCCmax', 'rlnCutOff', 'rlnSearchStd',
       'rlnTomoTiltSeriesPixelSize', 'rlnTomoName'],
      dtype='object')

In [21]:
df_star

,rlnCenteredCoordinateXAngst,rlnCenteredCoordinateYAngst,rlnCenteredCoordinateZAngst,rlnAngleRot,rlnAngleTilt,rlnAnglePsi,rlnLCCmax,rlnCutOff,rlnSearchStd,rlnTomoTiltSeriesPixelSize,rlnTomoName
0,416.64,618.24,-40.32,-6.923080,84.113158,71.538462,0.365649,0.214109,0.029137,13.44,pink_MGS007_T4_ts_006
1,2701.44,2768.64,-806.40,-24.230770,75.142834,-11.538460,0.357390,0.214109,0.029137,13.44,pink_MGS007_T4_ts_006
2,2553.60,362.88,-577.92,-41.538460,90.000000,20.769231,0.356250,0.214109,0.029137,13.44,pink_MGS007_T4_ts_006
3,2728.32,927.36,-201.60,-103.846150,95.886842,11.538462,0.352523,0.214109,0.029137,13.44,pink_MGS007_T4_ts_006
4,2056.32,-483.84,-483.84,-13.846150,84.113158,39.230769,0.350261,0.214109,0.029137,13.44,pink_MGS007_T4_ts_006
...,...,...,...,...,...,...,...,...,...,...,...
58529,134.40,2607.36,430.08,173.076923,90.000000,-34.615390,0.255740,0.250251,0.034056,13.44,pink3_MGS001_T2_ts_006
58530,1155.84,-1344.00,-40.32,148.846154,104.857166,-173.076920,0.255478,0.250251,0.034056,13.44,pink3_MGS001_T2_ts_006
58531,1008.00,2822.40,819.84,169.615385,92.939534,103.846154,0.255398,0.250251,0.034056,13.44,pink3_MGS001_T2_ts_006
58532,-1276.80,-779.52,-1464.96,34.615385,59.148114,-177.692310,0.255255,0.250251,0.034056,13.44,pink3_MGS001_T2_ts_006


In [22]:
#get unique and same order tomogram names 
tomo_names = df_star['rlnTomoName'].unique()
tomo_names


array(['pink_MGS007_T4_ts_006', 'pink_MGS007_T5_ts_006',
       'pink_MGS007_T2_ts_008', 'pink_MGS007_T5_ts_005',
       'pink_MGS007_T5_ts_004', 'pink_MGS007_T5_ts_001',
       'pink_MGS007_T5_ts_007', 'pink_MGS007_T5_ts_002',
       'pink_MGS007_T5_ts_003', 'pink_MGS007_T4_ts_001',
       'pink_MGS007_T2_ts_009', 'pink_MGS007_T4_ts_007',
       'MGS5_004_T2_ts_012', 'MGS5_004_T2_ts_008', 'MGS5_005_T1_ts_002',
       'MGS5_005_T1_ts_003', 'MGS5_004_T2_ts_007', 'MGS5_005_T1_ts_004',
       'MGS5_004_T2_ts_006', 'MGS5_004_T2_ts_013', 'MGS5_004_T2_ts_009',
       'MGS5_004_T2_ts_010', 'MGS5_004_T2_ts_005', 'MGS5_004_T2_ts_011',
       'MGS5_005_T1_ts_001', 'pink_MGS006_T2_ts_013',
       'pink_MGS006_T2_ts_006', 'pink_MGS006_T3_ts_002',
       'pink_MGS006_T2_ts_012', 'pink_MGS006_T2_ts_007',
       'pink_MGS006_T2_ts_008', 'pink_MGS006_T2_ts_010',
       'pink_MGS006_T3_ts_003', 'pink_MGS006_T2_ts_011',
       'pink_MGS006_T3_ts_001', 'pink_MGS006_T2_ts_014',
       'pink_MGS006_T2_ts_0

In [23]:
# read vll file
vll_path = '/home/muwang/Documents/projects/IR/20230518IR_mCherry/all_tomograms_bin8.txt'
vll_df = read_vll_to_df(vll_path)
vll_df.keys()

# 将vll_df中的mrc绝对路径（列名假设为 'path'）按照tomo_names重排并保存
import os



# 用法示例：
save_sorted_vll_by_tomonames(tomo_names, vll_df, "output/sorted_tomos_bin8.vll")



已根据tomo_names顺序保存vll到: output/sorted_tomos_bin8.vll
